In [8]:
import numpy as np
import itertools
import math

In [9]:
class SubBlockModel:
    def __init__(self, subblock):
        self.subblock = subblock
        self.M = self._init_M()
        self.M0 =  self.M.copy()
    
    # Initialise probability matrix M
    # M[i][j]: The probability of node i having value j
    def _init_M(self):
        M = np.full((9,9),1/9)
        for i,x in enumerate(self.subblock.flatten()):
            x = int(x)
            if x==-1:
                continue
            M[i] = [1 if i==x else 0 for i in range(1,10)]
        return M
    
    # Sample the probability distribution
    # Returns a 1D matrix: sample[i] is the probability distribution of node i over each value
    def sample(self):
        sample = np.empty(9)
        values = range(1,10)
        for i in range(9):
            sample[i] = np.random.choice(values,size=None,p=self.M[i])
        return sample
    
    # N: Number of samples (Population Size)
    # c: Sample Size (Half Good, Half Bad)
    def samples(self, N,c):
        n = math.floor(c/2)
        samples = [self.sample() for _ in range(N)]
        samples.sort(key=fitness,reverse=True)
        good_samples = [ delta(sample) for sample in samples[:n]]
        bad_samples = [ delta(sample) for sample in samples[-n:]]
        return good_samples, bad_samples

    # Update the model using good and bad samples
    # k: Learning Rate
    def update(self, k, delta_samples):
        good_samples, bad_samples = delta_samples
        self.M = self._update_good_samples(k,good_samples)
        self.M = self._update_bad_samples(k,bad_samples)

    def _update_good_samples(self, k, deltas):
        X1 = sum(deltas)
        X2 = np.empty((9,9))
        for i in range(9):
            rows_i = [ sample[i] for sample in deltas]
            X2[i] = sum(rows_i)
        return self.M  + (k/9) * X1 - (k/9**2)  * X2

    def _update_bad_samples(self, k, deltas):
        X1 = sum(deltas)
        X2 = np.empty((9,9))
        for i in range(9):
            rows_i = [ sample[i] for sample in deltas]
            X2[i] = sum(rows_i)
        return self.M  - (k/9) * X1 + (k/9**2)  * X2

# Sum of the number of UNIQUE values in each row and column (higher the fitness the better)
# 3x3 Subblock =>  18
def fitness(sample):
    sample = sample.reshape((3,3))
    row_fitness = 0
    col_fitness = 0
    for row in sample:
        row_fitness += len(set(row).difference({-1}))
    for col in sample.T:
        col_fitness += len(set(col).difference({-1}))
    return row_fitness + col_fitness

# Given a 1D sample, apply delta function
# Returns a 2D matrix: sample[i][j]=1 if the value of node i=j, otherwise 0
def delta(sample):
    res = np.empty((9,9))
    for i,x in enumerate(sample):
        x = int(x)
        res[i] = np.full(9,0)
        res[i][x-1] = 1
    return res

# Returns true if probability distribution of values sum to 1 for each cell
def check_M(M):
    return np.allclose(np.ones(9),np.sum(M,axis=1))

In [10]:
puzzle = np.loadtxt('puzzle.txt')
subblocks= [np.hsplit(x,3) for x in np.vsplit(puzzle,3)]
subblocks = list(itertools.chain.from_iterable(subblocks))

In [11]:
models = [SubBlockModel(block) for block in subblocks]
m0 = models[0]

In [12]:
N = 150
c = 50
k = 0.5

delta_samples = m0.samples(N,c)
m0.update(k, delta_samples)

In [13]:
m0.M0

array([[0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
        0.11111111, 0.11111111, 0.11111111, 0.11111111],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
        0.11111111, 0.11111111, 0.11111111, 0.11111111],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        ],
       [0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
        0.11111111, 0.11111111, 0.11111111, 0.11111111],


In [14]:
# Some probabilities are negative but sum of distributions=1
m0.M, check_M(m0.M)

(array([[ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.16049383,  0.16049383, -0.28395062,  0.25925926, -0.43209877,
          0.30864198,  0.40740741,  0.35802469,  0.0617284 ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  1.        ],
        [ 0.20987654,  0.30864198,  0.30864198, -0.03703704, -0.13580247,
          0.20987654,  0.30864198,  0.11111111, -0.28395062],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.45679012,  0.20987654,  0.11111111, 